In [ ]:
'''
To do:
5. merge api data back to streams (columns as lists for artist data)
7. reference ids to avoid dupe name issue
'''

In [ ]:
# Part 0: Install and import packages

# 0.1 Install necessary packages

!pip install pandas matplotlib seaborn spotipy requests
print("Installed all necessary packages.")

In [ ]:
# 0.2 Import needed libraries

import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import time
import sys
import requests
import base64
from urllib.parse import urlencode


print("Imported necessary libraries.")

In [ ]:
# Part 1: Load and clean data

# 1.1 For all accounts, combine all json files

# Move all uncompressed export folders into a folder titled 'rd'. 'rd' should exist beside this .ipynb file. 
# Keep all .json files inside account folder.

accounts = ['mmcmm', 'thing21', 'danDrust', 'michaelBesch', 'walter'] # Include names of all account folders you want to analyze

prefix = "Streaming_History"

alldata = []

for path in accounts:

    # Path to the folder containing JSON files
    folder_path = 'rd/' + path + '/'  
    
    # List all Streaming History JSON files in the folder
    pattern = f"{prefix}*.json"
    json_files = glob.glob(os.path.join(folder_path, pattern))
    
    # Initialize an empty list to hold DataFrames
    dataframes = []
    
    # Loop through each JSON file and read it into a DataFrame
    for file in json_files:
        try:
            df = pd.read_json(file)  # Read JSON file into a DataFrame
            dataframes.append(df)    # Append the DataFrame to the list
        except ValueError as e:
            print(f"Error reading {file}: {e}")
    
    # Concatenate all DataFrames into a single DataFrame
    final_df = pd.concat(dataframes, ignore_index=True)
    
    # Output the final DataFrame to a CSV file
    output_file = 'outputs/' + path + 'Streams.csv'  
    final_df.to_csv(output_file, index=False)
    
    print(f"{path} data saved as {output_file}")
    
    # Clean and transform account data for use
    final_df['ts'] = pd.to_datetime(final_df['ts'])
    final_df['skipped'] = final_df['skipped'].astype(bool)
    final_df['offline'] = final_df['offline'].astype(bool)
    final_df['incognito_mode'] = final_df['incognito_mode'].astype(bool)
    final_df['offline_timestamp'] = final_df['offline_timestamp'].replace([float('inf'), float('-inf')], None)
    final_df['offline_timestamp'] = final_df['offline_timestamp'].fillna(0)
    final_df['offline_timestamp'] = final_df['offline_timestamp'].astype(int)
    final_df['offline_timestamp'] = pd.to_datetime(final_df['offline_timestamp'], unit='s', errors='coerce')

    # Use alldata to collect data from all accounts
    alldata.append(final_df)

# Concatenate all DataFrames into a single DataFrame
streams = pd.concat(alldata, ignore_index=True)

# Output the final DataFrame to a CSV file
output_name = 'outputs/combined.csv'  
streams.to_csv(output_name, index=False)

print(f"Combined CSV of all streams saved as {output_name}. Used in code as streams.")

In [ ]:
# Part 2: Enhance data

# 2.1 Establish API connection

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = "" # message mitch for keys if wanted
client_secret = ""

# Set up authorization
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

token = auth_manager.get_access_token()

print(f"Established connection to Spotipy API. \nToken is {token}")

In [ ]:
# To do: Add in audio api

# 2.2 Prepare track library

# LOAD OR INIT TRACK LIBRARY
track_library_file = "cache/trackLibrary.csv"

try:
    track_library_df = pd.read_csv(track_library_file)
except FileNotFoundError:
    track_library_df = pd.DataFrame(columns=[
        'track_id', 'track_name', 'album_name', 'album_release_date', 'album_id', 'album_type',
        'artist_names', 'artist_ids', 'track_popularity', 'duration_ms', 'explicit', 'isrc',
        'ean', 'upc', 'external_urls', 'track_uri'
    ])

# IDENTIFY NEW TRACKS
track_uris = streams['spotify_track_uri'].dropna().astype(str).unique().tolist()
track_ids = [uri.split(':')[-1] for uri in track_uris if 'spotify:track:' in uri]
all_track_ids = set(track_ids)

known_track_ids = set(track_library_df['track_id'].dropna().unique())
new_track_ids = all_track_ids - known_track_ids

print(f"{len(all_track_ids)} unique track IDs in streams.")
print(f"{len(known_track_ids)} tracks in track cache.")
print(f"{len(new_track_ids)} new tracks to fetch.")

# FETCH TRACK DETAILS
tracks_url = "https://api.spotify.com/v1/tracks"
token_str = token['access_token'] 
headers = {"Authorization": f"Bearer {token_str}"}

def chunked(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

new_tracks_data = []
for i, chunk in enumerate(chunked(list(new_track_ids), 50)):
    params = {"ids": ",".join(chunk)}
    r = requests.get(tracks_url, headers=headers, params=params)
    if r.status_code != 200:
        print(f"Request failed with status {r.status_code}, waiting 30s before retrying...")
        time.sleep(30)
        continue
    response_data = r.json()
    tracks = response_data.get('tracks', [])
    for t in tracks:
        if t:
            new_tracks_data.append({
                "track_id": t.get("id"),
                "track_name": t.get("name"),
                "album_name": t.get("album", {}).get("name"),
                "album_release_date": t.get("album", {}).get("release_date"),
                "album_id": t.get("album", {}).get("id"),
                "album_type": t.get("album", {}).get("album_type"),
                "artist_names": [artist["name"] for artist in t.get("artists", [])],
                "artist_ids": [artist["id"] for artist in t.get("artists", [])],
                "track_popularity": t.get("popularity"),
                "duration_ms": t.get("duration_ms"),
                "explicit": t.get("explicit"),
                "isrc": t.get("external_ids", {}).get("isrc"),
                "ean": t.get("external_ids", {}).get("ean"),
                "upc": t.get("external_ids", {}).get("upc"),
                "external_urls": t.get("external_urls", {}),
                "track_uri": t.get("uri")
            })
    # checkpoint counts and small delay to reduce rate limit risks
    if i % 10 == 0 and i > 0:
        print(f"Fetched metadata for {i * 50} tracks.")
    time.sleep(0.2)

print("Completed track api calls.")
new_tracks_df = pd.DataFrame(new_tracks_data)
print(f"Fetched metadata for {len(new_tracks_df)} total new tracks.")

if new_tracks_data:
    # Append to track_library_df
    track_library_df = pd.concat([track_library_df, new_tracks_df], ignore_index=True)
    # Remove duplicates just in case
    track_library_df.drop_duplicates(subset=['track_id'], inplace=True)
    track_library_df.to_csv(track_library_file, index=False)
    print(f"Added {len(new_tracks_data)} new albums to {track_library_file}")


In [ ]:
# Deprecated: Ignore this cell

# 2.25 Prepare audio library

# LOAD OR INIT AUDIO LIBRARY
audio_library_file = "cache/audioLibrary.csv"

try:
    audio_library_df = pd.read_csv(audio_library_file)
except FileNotFoundError:
    audio_library_df = pd.DataFrame(columns=[
        'track_id', 'duration_ms', 'analysis_url', 'acousticness',  'danceability',  'energy',
        'instrumenalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness',
        'tempo', 'time_signature', 'valence'
    ])

# IDENTIFY NEW AUDIO INFO NEEDED
track_uris = streams['spotify_track_uri'].dropna().astype(str).unique().tolist()
track_ids = [uri.split(':')[-1] for uri in track_uris if 'spotify:track:' in uri]
all_track_ids = set(track_ids)

known_track_ids = set(audio_library_df['track_id'].dropna().unique())
new_track_ids = all_track_ids - known_track_ids

print(f"{len(all_track_ids)} unique track IDs in streams.")
print(f"{len(known_track_ids)} tracks in audio cache.")
print(f"{len(new_track_ids)} new tracks to fetch audio info.")

# FETCH AUDIO DETAILS
tracks_url = "https://api.spotify.com/v1/audio-features"
token_str = token['access_token'] 
headers = {"Authorization": f"Bearer {token_str}"}

def chunked(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

new_tracks_data = []
for i, chunk in enumerate(chunked(list(new_track_ids), 50)):
    params = {"ids": ",".join(chunk)}
    r = requests.get(tracks_url, headers=headers, params=params)
    if r.status_code != 200:
        print(f"Request failed with status {r.status_code}, waiting 30s before retrying...")
        time.sleep(30)
        continue
    response_data = r.json()
    tracks = response_data.get('tracks', [])
    for t in tracks:
        if t:
            new_tracks_data.append({
                "track_id": t.get("id"),
                "duration_ms": t.get("duration_ms"),
                "analysis_url": t.get("analysis_url"),
                "acousticness": t.get("acousticness"),
                "danceability": t.get("danceability"),
                "energy": t.get("energy"),
                "instrumenalness": t.get("instrumenalness"),
                "key": t.get("key"),
                "liveness": t.get("liveness"),
                "loudness": t.get("loudness"),
                "mode": t.get("mode"),
                "speechiness": t.get("speechiness"),
                "tempo": t.get("tempo"),
                "time_signature": t.get("time_signature"),
                "valence": t.get("valence")
            })
    # checkpoint counts and small delay to reduce rate limit risks
    if i % 10 == 0 and i > 0:
        print(f"Fetched metadata for {i * 50} tracks.")
    time.sleep(0.2)

print("Completed track api calls.")
new_tracks_df = pd.DataFrame(new_tracks_data)
print(f"Fetched audio features for {len(new_tracks_df)} total new tracks.")

if new_tracks_data:
    # Append to audio_library_df
    audio_library_df = pd.concat([audio_library_df, new_tracks_df], ignore_index=True)
    # Remove duplicates just in case
    audio_library_df.drop_duplicates(subset=['track_id'], inplace=True)
    audio_library_df.to_csv(audio_library_file, index=False)
    print(f"Added {len(new_tracks_data)} new albums to {audio_library_file}")

In [ ]:
# 2.3 Prepare album library

# LOAD OR INIT ALBUM LIBRARY
album_library_file = "cache/albumLibrary.csv"

try:
    album_library_df = pd.read_csv(album_library_file)
except FileNotFoundError:
    album_library_df = pd.DataFrame(columns=[
        'album_id', 'album_name', 'album_type', 'total_tracks', 'release_date', 
        'release_date_precision', 'label', 'album_popularity', 'available_markets',
        'album_uri', 'album_external_urls', 'image'
    ])

# IDENTIFY NEW ALBUMS
all_album_ids = set(track_library_df['album_id'].dropna().unique())
known_album_ids = set(album_library_df['album_id'].dropna().unique())
new_album_ids = all_album_ids - known_album_ids

print(f"{len(all_album_ids)} unique album IDs in track library.")
print(f"{len(known_album_ids)} albums in album cache.")
print(f"{len(new_album_ids)} new albums to fetch.")

# FETCH ALBUM DETAILS
albums_url = "https://api.spotify.com/v1/albums"
token_str = token['access_token'] 
headers = {"Authorization": f"Bearer {token_str}"}

new_albums_data = []
for i, chunk in enumerate(chunked(list(new_album_ids), 20)):
    params = {"ids": ",".join(chunk)}
    r = requests.get(albums_url, headers=headers, params=params)
    if r.status_code != 200:
        print(f"Albums request failed with {r.status_code}, waiting 30s...")
        time.sleep(30)
        continue
    albums_json = r.json().get('albums', [])
    for alb in albums_json:
        if alb:
            new_albums_data.append({
                "album_id": alb.get('id'),
                "album_name": alb.get('name'),
                "album_type": alb.get('album_type'),
                "total_tracks": alb.get('total_tracks'),
                "release_date": alb.get('release_date'),
                "release_date_precision": alb.get('release_date_precision'),
                "label": alb.get('label'),
                "album_popularity": alb.get('popularity'),
                "available_markets": alb.get('available_markets'),
                "album_uri": alb.get('uri'),
                "album_external_urls": alb.get('external_urls', {}),
                'image': alb.get("images", {})
            })
    # checkpoint counts and small delay to reduce rate limit risks
    if i % 25 == 0 and i > 0:
        print(f"Fetched metadata for {i * 20} albums.")
    time.sleep(0.2)

print("Completed album api calls.")
new_albums_df = pd.DataFrame(new_albums_data)
print(f"Fetched metadata for {len(new_albums_df)} total new albums.")

if new_albums_data:
    # Append to album_library_df
    album_library_df = pd.concat([album_library_df, new_albums_df], ignore_index=True)
    # Remove duplicates just in case
    album_library_df.drop_duplicates(subset=['album_id'], inplace=True)
    album_library_df.to_csv(album_library_file, index=False)
    print(f"Added {len(new_albums_data)} new albums to {album_library_file}")


In [ ]:
# 2.4 Prepare artist library

# LOAD OR INIT ARTIST LIBRARY
artist_library_file = "cache/artistLibrary.csv"

try:
    artist_library_df = pd.read_csv(artist_library_file)
except FileNotFoundError:
    artist_library_df = pd.DataFrame(columns=[
        'artist_id', 'artist_name', 'artist_popularity', 'artist_followers',
        'artist_genres', 'artist_uri', 'artist_external_urls'
    ])

# IDENTIFY NEW ARTISTS
all_artist_ids = set()
for artist_list in track_library_df['artist_ids'].dropna():
    if isinstance(artist_list, str):
        import ast
        artist_list = ast.literal_eval(artist_list)  # convert string "[...]" to Python list
    all_artist_ids.update(artist_list)

known_artist_ids = set(artist_library_df['artist_id'].dropna().unique())
new_artist_ids = all_artist_ids - known_artist_ids

print(f"{len(all_artist_ids)} unique artist IDs in track library.")
print(f"{len(known_artist_ids)} artists in artist cache.")
print(f"{len(new_artist_ids)} new artists to fetch.")

# FETCH ARTIST DETAILS
artists_url = "https://api.spotify.com/v1/artists"
new_artists_data = []
for i, chunk in enumerate(chunked(list(new_artist_ids), 50)):
    params = {"ids": ",".join(chunk)}
    r = requests.get(artists_url, headers=headers, params=params)
    if r.status_code != 200:
        print(f"Artists request failed with {r.status_code}, waiting 30s...")
        time.sleep(30)
        continue
    artists_json = r.json().get('artists', [])
    for art in artists_json:
        if art:
            new_artists_data.append({
                "artist_id": art.get('id'),
                "artist_name": art.get('name'),
                "artist_popularity": art.get('popularity'),
                "artist_followers": art.get('followers', {}).get('total'),
                "artist_genres": art.get('genres', []),
                "artist_uri": art.get('uri'),
                "artist_external_urls": art.get('external_urls', {})
            })
    # checkpoint counts and small delay to reduce rate limit risks
    if i % 10 == 0 and i > 0:
        print(f"Fetched metadata for {i * 50} artists.")
    time.sleep(0.2)

print("Completed artist api calls.")
new_artists_df = pd.DataFrame(new_artists_data)
print(f"Fetched metadata for {len(new_artists_df)} total new artists.")

if new_artists_data:
    # Append to artist_library_df
    artist_library_df = pd.concat([artist_library_df, new_artists_df], ignore_index=True)
    # Remove duplicates just in case
    artist_library_df.drop_duplicates(subset=['artist_id'], inplace=True)
    artist_library_df.to_csv(artist_library_file, index=False)
    print(f"Added {len(new_artists_data)} new artists to {artist_library_file}")


In [ ]:
# 2.5 Merge streams with libraries

# Ensure streams has a track_id column for merging
streams['track_id'] = streams['spotify_track_uri'].astype(str).apply(
    lambda x: x.split(':')[-1] if 'spotify:track:' in x else x
)

with_track_info = streams.merge(track_library_df, on='track_id', how='left')
with_album_info = with_track_info.merge(album_library_df, on='album_id', how='left')
# with_artist_info = with_album_info.merge(artist_library_df, on='artist_id', how='left') # multi-artist will make this an issue
# with_audio_info = with_artist_info.merge(audio_library_df, on='track_id', how='left')

print("Enriched streams DataFrame:")
display(with_album_info.head())

In [ ]:
# Part 3: Graph and display data insights

# 3.1 Lifetime rankings by streams

# Top Artists by Play Count
top_artists = streams['master_metadata_album_artist_name'].value_counts().head(20)
# print("Top 20 Artists by Play Count:")
# print(top_artists)

plt.figure(figsize=(10,6))
top_artists.plot(kind='bar')
plt.title("Top 20 Artists by Number of Streams")
plt.show()

# Top Albums by Play Count
top_albums = streams['master_metadata_album_album_name'].value_counts().head(20)
# print("Top 20 Albums by Play Count:")
# print(top_albums)

plt.figure(figsize=(10,6))
top_albums.plot(kind='bar')
plt.title("Top 20 Albums by Number of Streams")
plt.show()

# Top Songs by Play Count
top_songs = streams['master_metadata_track_name'].value_counts().head(20)
# print("Top 20 Songs by Play Count:")
# print(top_songs)

plt.figure(figsize=(10,6))
top_songs.plot(kind='bar')
plt.title("Top 20 Songs by Number of Streams")
plt.show()

In [ ]:
# 3.2 Lifetime rankings by duration

# Top Artists by Total Play Duration
top_artists = streams.groupby('master_metadata_album_artist_name')['ms_played'].sum().div(3600000).sort_values(ascending=False).head(20)
# print("Top 20 Artists by Total Play Duration (hours):")
# print(top_artists)

plt.figure(figsize=(10, 6))
top_artists.plot(kind='bar')
plt.title("Top 20 Artists by Total Play Duration")
plt.xlabel("Artist")
plt.ylabel("Play Duration (hours)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Top Albums by Total Play Duration
top_albums = streams.groupby('master_metadata_album_album_name')['ms_played'].sum().div(3600000).sort_values(ascending=False).head(20)
# print("Top 20 Albums by Total Play Duration (hours):")
# print(top_albums)

plt.figure(figsize=(10, 6))
top_albums.plot(kind='bar')
plt.title("Top 20 Albums by Total Play Duration")
plt.xlabel("Album")
plt.ylabel("Play Duration (hours)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Top Songs by Total Play Duration
top_songs = streams.groupby('master_metadata_track_name')['ms_played'].sum().div(3600000).sort_values(ascending=False).head(20)
# print("Top 20 Songs by Total Play Duration (hours):")
# print(top_songs)

plt.figure(figsize=(10, 6))
top_songs.plot(kind='bar')
plt.title("Top 20 Songs by Total Play Duration")
plt.xlabel("Song")
plt.ylabel("Play Duration (hours)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
# 3.3 Lifetime listening durations

# Create a helper column for hours played
streams['hours_played'] = streams['ms_played'] / (1000 * 60 * 60)  # Convert ms to hours

# Set the index to the datetime column so resample works cleanly
streams.set_index('ts', inplace=True)

# --- For Number of Streams ---

# Resample to monthly frequency, counting the number of streams per month
monthly_counts = streams.resample('ME').size()

# Calculate 12-month rolling average of the monthly counts
counts_12m_avg = monthly_counts.rolling(12).mean()

# --- For Total Duration ---

# Resample to monthly frequency, summing the hours played per month
monthly_duration = streams['hours_played'].resample('ME').sum()

# Calculate 12-month rolling average of monthly duration
duration_12m_avg = monthly_duration.rolling(12).mean()

# --- Plotting ---

plt.figure(figsize=(10, 6))

# Plot the 12-month rolling average of number of streams
ax = counts_12m_avg.plot(label='12-Month Rolling Avg of Streams', color='blue')

# Secondary axis for total duration
ax2 = ax.twinx()
duration_12m_avg.plot(ax=ax2, label='12-Month Rolling Avg of Duration (hours)', color='green', linestyle='--')

# Labeling
ax.set_xlabel('Date')
ax.set_ylabel('Number of Streams')
ax2.set_ylabel('Total Duration (hours)')
plt.title("12-Month Rolling Average: Number of Streams & Total Play Duration")

# Combine legends
lines_1, labels_1 = ax.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax2.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# 3.4 Top 10 artists over time

# Convert ms to hours if not already done
streams['hours_played'] = streams['ms_played'] / (1000.0 * 60 * 60)

# Identify the top 10 artists
artist_totals = streams.groupby('master_metadata_album_artist_name')['hours_played'].sum()
top10_artists = artist_totals.nlargest(10).index

# Filter to those artists
top10_data = streams[streams['master_metadata_album_artist_name'].isin(top10_artists)]

# Group by month & artist, sum hours_played
monthly_artist = top10_data.groupby([pd.Grouper(freq='ME'), 'master_metadata_album_artist_name'])['hours_played'].sum()
monthly_artist_pivot = monthly_artist.unstack('master_metadata_album_artist_name')

# Compute 12-month rolling average
monthly_artist_12m = monthly_artist_pivot.rolling(12).mean()

# Plot
plt.figure(figsize=(10, 6))
monthly_artist_12m.plot(ax=plt.gca())
plt.title("12-Month Rolling Avg of Hours Played for Top 10 Artists")
plt.xlabel("Date")
plt.ylabel("Hours Played (Monthly)")
plt.legend(title="Artist")
plt.tight_layout()
plt.show()

In [ ]:
# 3.5 Custom artist line

# Convert ms to hours if not already done
streams['hours_played'] = streams['ms_played'] / (1000.0 * 60 * 60)

artists = ['Attack Attack!']
# Filter to those artists
top10_data = streams[streams['master_metadata_album_artist_name'].isin(artists)]

# Group by month & artist, sum hours_played
monthly_artist = top10_data.groupby([pd.Grouper(freq='ME'), 'master_metadata_album_artist_name'])['hours_played'].sum()
monthly_artist_pivot = monthly_artist.unstack('master_metadata_album_artist_name')

# Compute 12-month rolling average
monthly_artist_12m = monthly_artist_pivot.rolling(12).mean()

# Plot
plt.figure(figsize=(10, 6))
monthly_artist_12m.plot(ax=plt.gca())
plt.title("12-Month Rolling Avg of Hours Played for Top 10 Artists")
plt.xlabel("Date")
plt.ylabel("Hours Played (Monthly)")
plt.legend(title="Artist")
plt.tight_layout()
plt.show()

In [ ]:
# 3.6 Monthly topline

streams['month'] = streams.index.to_period('M')

# Group by month and collect results in a list
summary_rows = []
for month_period, group_df in streams.groupby('month'):
    # Total stream time (hours) for this month
    total_stream_time = group_df['hours_played'].sum()

    # Top artist for this month (by total hours)
    # Group by artist, sum hours_played, pick the max
    artist_sums = group_df.groupby('master_metadata_album_artist_name')['hours_played'].sum().sort_values(ascending=False)
    top_artist = artist_sums.index[0] if not artist_sums.empty else None

    # Top album for this month
    album_sums = group_df.groupby('master_metadata_album_album_name')['hours_played'].sum().sort_values(ascending=False)
    top_album = album_sums.index[0] if not album_sums.empty else None

    # Top song for this month
    song_sums = group_df.groupby('master_metadata_track_name')['hours_played'].sum().sort_values(ascending=False)
    top_song = song_sums.index[0] if not song_sums.empty else None

    summary_rows.append([
        month_period,
        top_artist,
        top_album,
        top_song,
        total_stream_time
    ])

# Create a summary DataFrame from the rows
summary_df = pd.DataFrame(summary_rows, columns=[
    'month',
    'top_artist',
    'top_album',
    'top_song',
    'total_stream_time_hours'
])

# Convert the 'month' Period to a timestamp (end of month) for clarity, or keep as Period
summary_df['month'] = summary_df['month'].dt.to_timestamp(how='end')

# Sort by month chronologically
summary_df.sort_values('month', inplace=True)
summary_df.reset_index(drop=True, inplace=True)

# Remove limits on rows and columns shown
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Now any display(df) call will show all rows and columns
display(summary_df)

# Reset to defaults limits
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
